# Информация о географическом расположении клиентов (geolocation_olist_public_dataset.csv)

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys, os
import pathlib
from pathlib import Path
from olist_churn_prediction.paths import SRC_DIR, PROCESSED_DIR, INTERIM_NOTEBOOK_DIR, RAW_DIR
from olist_churn_prediction import feature_processing

In [3]:
geolocation_data = pd.read_csv(RAW_DIR / "geolocation_olist_public_dataset.csv")

In [4]:
display(geolocation_data.head())
print(geolocation_data.shape)
geolocation_data.dtypes

,zip_code_prefix,city,state,lat,lng
0,10,sao paulo,sp,-23.547807,-46.636122
1,10,sao paulo,sp,-23.546081,-46.644820
2,10,sao paulo,sp,-23.540984,-46.642569
3,10,sao paulo,sp,-23.546758,-46.645771
4,10,sao paulo,sp,-23.546362,-46.643073


(323016, 5)


zip_code_prefix      int64
city                object
state               object
lat                float64
lng                float64
dtype: object

#### Пояснения к признакам:

`zip_code_prefix` - это префикс почтового индекса. До 3 цифр;

`city` - город заказчика;

`state` - штат заказчика;

`lat` - анг. latitude, широта. Координаты ненастоящие, созданы случайно на основе принадлежности к определенному zip_code_prefix;

`lng` - анг. longitude, долгота. Также ненастоящие.

In [5]:
print('Пропуски:', geolocation_data.isna().sum())

Пропуски: zip_code_prefix    0
city               0
state              0
lat                0
lng                0
dtype: int64


Приведем значения к нижнему регистру:

In [6]:
df = geolocation_data.copy()

In [7]:
feature_processing.lowercase_categoricals(df, cat_cols=['city', 'state'], inplace=True)

,zip_code_prefix,city,state,lat,lng
0,10,sao_paulo,sp,-23.547807,-46.636122
1,10,sao_paulo,sp,-23.546081,-46.644820
2,10,sao_paulo,sp,-23.540984,-46.642569
3,10,sao_paulo,sp,-23.546758,-46.645771
4,10,sao_paulo,sp,-23.546362,-46.643073
...,...,...,...,...,...
323011,999,muliterno,rs,-28.329718,-51.769615
323012,999,tapejara,rs,-28.071667,-52.014481
323013,999,ipiranga_do_sul,rs,-27.935243,-52.426996
323014,999,ciriaco,rs,-28.343103,-51.870397


Обработка проблемных городов:

In [8]:
feature_processing.disambiguate_city_state(df, city_col="city", state_col="state", inplace=True)

,zip_code_prefix,city,state,lat,lng
0,10,sao_paulo,sp,-23.547807,-46.636122
1,10,sao_paulo,sp,-23.546081,-46.644820
2,10,sao_paulo,sp,-23.540984,-46.642569
3,10,sao_paulo,sp,-23.546758,-46.645771
4,10,sao_paulo,sp,-23.546362,-46.643073
...,...,...,...,...,...
323011,999,muliterno,rs,-28.329718,-51.769615
323012,999,tapejara_rs,rs,-28.071667,-52.014481
323013,999,ipiranga_do_sul,rs,-27.935243,-52.426996
323014,999,ciriaco,rs,-28.343103,-51.870397


In [9]:
for col in df.columns.drop(['lat', 'lng']):
    print(f"--- {col} ---")
    print(df[col].value_counts())
    print()

--- zip_code_prefix ---
zip_code_prefix
997    500
10     500
11     500
12     500
13     500
      ... 
947      4
300      2
704      1
325      1
907      1
Name: count, Length: 851, dtype: int64

--- city ---
city
sao_paulo                25001
curitiba                 11605
rio_de_janeiro           10020
belo_horizonte            8500
porto_alegre              7357
                         ...  
general_sampaio              1
caponga                      1
general_carneiro_mt          1
novo_santo_antonio_mt        1
sao_jose_do_povo             1
Name: count, Length: 5730, dtype: Int64

--- state ---
state
sp    81929
mg    39851
rs    34831
pr    31441
rj    29057
ba    20463
go    11497
pe    11460
sc     9342
ce     7690
df     5645
pa     5237
ms     4921
es     4421
mt     4387
ma     3823
pb     2847
rn     2640
pi     2509
to     2394
al     1875
se     1448
ro     1001
am      807
ac      500
ap      500
rr      500
Name: count, dtype: Int64



In [10]:
print(len(geolocation_data['city'].unique()))
print(len(geolocation_data['state'].unique()))

5419
27


# Выводы из таблицы geolocation data:
1. Из-за отсутствия `order_id` или `customer_unique_id` будет затруднительно сопоставить местоположение клиента и его заказы. Возможно в будущем найдется способ это сделать.
2. В таблице 5419 уникальных городов и 27 уникальных штатов.
3. Возможно добаления новых признаков, основанных на расстоянии между продавцом и покупателем, хотя пока что нам неизвестны id покупателей.

In [12]:
df.to_parquet(INTERIM_NOTEBOOK_DIR / "geolocation_olist_public_dataset_interim.parquet", index=False)